### Imports

In [1]:
### Standard imports
import os
import json
import pickle
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from glob import glob
from os.path import join, isfile
from random import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU, ReLU
from tqdm import tqdm_notebook as tqdm

tf.random.set_seed(5)
random.seed(5)
np.random.seed(5)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

root = "/Users/pstetz/Desktop/confidential/.project"

### Setup

In [2]:
def save_history(history_path, data):
    if isfile(history_path):
        pd.concat([pd.read_csv(history_path), pd.DataFrame(data)]).to_csv(history_path, index=False)
    else:
        pd.DataFrame(data).to_csv(history_path, index=False)
        
with open("/Users/pstetz/Desktop/confidential/.project/summary/info_order.json", "r") as f:
    renaming = json.load(f)
renaming = {int(k): v for k, v in renaming.items()}

### Model

<div hidden>
def _load_model():
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
    activation = relu
    use_bias = False
    def layer_a(dim):
        return layers.Dense(dim, activation=activation, use_bias=use_bias)
    
    info_input = keras.Input(shape=(130,), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.MaxPool3D(),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
#     for dim in (130, 130, 130):
#         info_s.append(layer_a(dim)(info_s[-1]))

    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
#     for dim in (146, 146, 146, 146, 128, 64, 32):
    for dim in (258, 258, 128, 128, 64, 32):
        x_s.append(layer_a(dim)(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-4
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-2, decay=learning_rate/30),
      loss={"bold_signal": "mse"},
      loss_weights=[1.])
    return model
</div>

In [3]:
def _load_model(with_lgbm = False):
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
#     activation = lr
    
    info_input = keras.Input(shape=(130 + int(with_lgbm),), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
    init_shape = x_0.shape[1]
    for layer in (
        layers.Reshape((init_shape, 1)),
        layers.Conv1D(256, 256, strides=init_shape, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((256, 1)),
        layers.Conv1D(256, 256, strides=init_shape, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((256, 1)),
        layers.Conv1D(128, 256, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((128, 1)),
        layers.Conv1D(64, 128, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((64, 1)),
        layers.Conv1D(32, 64, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((32, 1)),
        layers.AveragePooling1D(2),
        layers.Flatten(),
    ):
        x_s.append(layer(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-3
    model.compile(
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-1, decay=learning_rate/30),
        loss={"bold_signal": "mse"}, loss_weights=[1.]
    )
    return model

if "model" in locals(): del model
model = _load_model()

### Setup

In [7]:
history_path = join(root, "dual_history.csv")
training_path = join(root, "training")
log_dir = join(root, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

train_batch  = 128
input_batch  = train_batch * 2
num_epoches = 2

if isfile(history_path):
    os.remove(history_path)

### Try double model (lgbm + CNN)

In [8]:
def create_lgbm_input(batch, renaming):
    df = pd.DataFrame(batch["info"])
    df.rename(columns=renaming, inplace=True)
    df["prev"] = batch["prev"][:, 4, 4, 4, 0]
    df["next"] = batch["next"][:, 4, 4, 4, 0]
    df["grey"] = batch["next"][:, 4, 4, 4, 1]
    return df

lgbm_path = "/Users/pstetz/Desktop/confidential/.project/run/lgbm/2_norm.pkl"

with open(lgbm_path, "rb") as f:
    lgbm_model = pickle.load(f)
    
model = _load_model(with_lgbm=True)

In [8]:
bold_signal, prev_volume, next_volume, info = [], [], [], []
for _ in range(4):
    training = glob(join(training_path, "*", "*"))
    shuffle(training)
    for i, batch_path in enumerate(tqdm(training)):
        if not isfile(join(batch_path, "norm_info.npy")):
            print("%s not found" % batch_path)
            continue
        bold_signal.extend(np.load(join(batch_path, "pred.npy")))
        prev_volume.extend(np.load(join(batch_path, "norm_prev.npy")))
        next_volume.extend(np.load(join(batch_path, "norm_next.npy")))
        info.extend(np.load(join(batch_path, "norm_info.npy"), allow_pickle=True))
        if (i + 1) % input_batch == 0:
            batch = {
                "prev": np.array(prev_volume),
                "next": np.array(next_volume),
                "info": np.array(info, dtype=np.float32)
            }
            lgbm_input = create_lgbm_input(batch, renaming)
            lgbm_preds = lgbm_model.predict(lgbm_input).reshape(len(batch["info"]), 1)
            batch["info"] = np.hstack((batch["info"], lgbm_preds))
            bold_signal = np.array(bold_signal)
            history = model.fit(
                batch, bold_signal,
                epochs=num_epoches, batch_size=train_batch, verbose=True,
                callbacks=[tensorboard_callback],
                shuffle=True # use when randomly selecting batches
            )
            _std = np.std(bold_signal).round(3)
            _loss = history.history["loss"][-1].round(3)
            assert not np.isnan(_loss), batch_path
            _mean = np.add(batch["prev"][:, 4, 4, 4, 0], batch["next"][:, 4, 4, 4, 0]) / 2
            mean_loss = np.sum(np.square(np.subtract(_mean, bold_signal))) / len(bold_signal)
            print(_loss, mean_loss)
            data = {
                "std": [_std] * num_epoches,
                "loss": history.history["loss"],
                "mean_loss": mean_loss
            }
            save_history(history_path, data)
            bold_signal, prev_volume, next_volume, info = [], [], [], []

/Users/pstetz/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Train on 32768 samples
Epoch 1/2
32768/32768 [==============================] - 23s 704us/sample - loss: 0.7963
Epoch 2/2
32768/32768 [==============================] - 20s 617us/sample - loss: 0.7434
0.743 0.7524904012680054
Train on 32768 samples
Epoch 1/2
32768/32768 [==============================] - 19s 567us/sample - loss: 0.7289
Epoch 2/2
32768/32768 [==============================] - 19s 586us/sample - loss: 0.7197
0.72 0.7295564413070679
Train on 32768 samples
Epoch 1/2
32768/32768 [==============================] - 18s 557us/sample - loss: 0.7353
Epoch 2/2
32768/32768 [==============================] - 16s 499us/sample - loss: 0.7270
0.727 0.7439282536506653
Train on 32768 samples
Epoch 1/2
32768/32768 [==============================] - 20s 599us/sample - loss: 0.7196
Epoch 2/2
32768/32768 [==============================] - 17s 531us/sample - loss: 0.7150
0.715 0.7309716939926147
Train on 32768 samples
Epoch 1/2
32768/32768 [==============================] - 15s 462us/sample 


KeyboardInterrupt



KeyboardInterrupt: 